In [ ]:
# Topic modeling uses statistical models to discover the topics present in a text or group of texts.
# We're going to use the LDA (Latent Dirichlet Allocation) approach for topic modeling in this module.
# But first, we're going to explore some visualization techniques we can apply to our text.

In [ ]:
# First, we'll load our CSV, look at the data and see what information we can glean from it,
# then create a bar graph of the parts of speech in the CSV we created earlier.

In [ ]:
import pandas
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt

# Open our CSV and read the data
document = pandas.read_csv('text_data.csv')

# Print the first 5 lines of our document to check that it loaded properly
print(document.head())

# Detect our Parts of Speech ('pos') column in the CSV
pos_group = document.groupby('pos')

# Print a summary statistic of all parts of speech grouped together
print(pos_group.describe(include='all'))

In [ ]:
# Now that we've learned how to manipulate the data a bit, we'll create a wordcloud with our data

In [ ]:
# Join the different words in our CSV into one long string
long_string = ','.join(list(document.text.values))

# Set up our wordcloud parameters
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                min_font_size = 10).generate(long_string) 
  
# create the wordcloud as an image                   
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
# Finally, we'll graph the 10 most common words in the text

In [ ]:
from collections import Counter
import numpy as np

# Set the text of our document as the word list
word_list = list(document.text.values)

# Get the 10 most common words from the list
counts = dict(Counter(word_list).most_common(10))

# Set up our labels
labels, values = zip(*counts.items())

# Sort our values in descending order
indSort = np.argsort(values)[::-1]

# Rearrange our data
labels = np.array(labels)[indSort]
values = np.array(values)[indSort]

indexes = np.arange(len(labels))

bar_width = 0.35

plt.bar(indexes, values)

# Add our labels
plt.xticks(indexes + bar_width, labels)
plt.show()

In [ ]:
# Now we'll use LDA model in SciKit Learn to conduct our topic modeling
# You can read more about topic modeling in Python at this link: 
# https://medium.com/mlreview/topic-modeling-with-scikit-learn-e80d33668730
# The article above also provides code on how to use SciKit Learn to perform
# NMF topic modeling on a text, which might be used and compared to our LDA results.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Load our document
document = pandas.read_csv('text_data.csv')

# Set our tokens as the 'lemma' column in our CSV
tokens = document.lemma.values

# SciKit Learn setup
documents = tokens

# Our number of topics and features
# Let's say we want 10 topics from the text
no_topics = 20

# The number of top words we'll include in our corpus 
no_features = 20000

# Setting up our LDA settings...
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features)
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()


# Running the LDA model
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 10
print(lda, tf_feature_names, no_top_words)